# Evaluation

This notebook contains all code to reproduce the results of our paper: "Using Modular Neural Networks for Anomaly Detection in Cyber-Physical Systems". 

In [ ]:
from eval_functions import *



In [ ]:
MODEL_LIST = ["MonolithAE", "MonolithVAE", "Modular0AE", "Modular0VAE", "Modular1AE", "Modular1VAE", "Modular2AE",  "Modular2VAE", "Modular3AE", "Modular3VAE"]
CASES = ["normal", "weight", "drop", "stop", "out", "push", "speed", "wrench"]
EVAL_PATH = "../logs/repl_studies/logs/train/"

RESULTS = ImportResults(path=EVAL_PATH, model_list=MODEL_LIST, anomaly_list=CASES)

## H1 Learning Individual Subsystems with MNNs

Evaluating for H1: Comparing the performance of the individual models for learning individual subsystems. 

In [ ]:
h1_res = H1(results_dict=RESULTS.glob_res)

In [ ]:
df_h1 = h1_res.df_joints

In [ ]:
# arrange the results for the paper: 
def arrange_df_h1(df_h1, model_list):
    df_h1 = df_h1.transpose()

    order = [col for col in model_list if col in df_h1.columns]
    df_h1 = df_h1[order]
    
    df1 = df_h1.iloc[::2].reset_index(drop=True)
    df2 = df_h1.iloc[1::2].reset_index(drop=True)
    
    
    # creat new df for merge
    df_h1_joints = pd.DataFrame(index=df1.index)
    
    for i, col in enumerate(df1.columns):
        df_h1_joints[f"{col} mean"] = df1[col]
        if i < len(df2.columns):
            df_h1_joints[f"{col} var"] = df2.iloc[:, i]
    return df_h1_joints 

df_h1_joints = arrange_df_h1(df_h1, MODEL_LIST)

tab_df_h1_joints = make_latex_table(df_h1_joints, decimals=3)
print(tab_df_h1_joints)

## H2 Detecting Anomalies Globally with MNNs

Evaluating the F1 for classifying anomalies from the testset and individual anomaly cases. The threshold for detecting anomalies is $2 \sigma$. 

In [ ]:
h2_res = H2(results_dict=RESULTS.glob_res, anomaly_list=CASES)

h2_res.df_model

In [ ]:
df_h2 = h2_res.df_anomaly

order = []
for col1 in MODEL_LIST: 
    for col2 in df_h2.columns:
        if col1 in col2:
            order.append(col2)
df_h2 = df_h2[order]

#print(df_h2)

tab_df_h2 = make_latex_table(df_h2, decimals=3)
print(tab_df_h2)

 ## H3 Allocating Anomaly root causes with MNNs

Evaluating the ability of detecting individual anomalies, and the root cause anomaly. 

In [ ]:
h3_res = H3(results_dict=RESULTS.glob_res)
df_h3_res = h3_res.df

order = []
for col1 in MODEL_LIST: 
    for col2 in df_h3_res.columns:
        if col1 in col2:
            order.append(col2)
df_h3_res = df_h3_res[order]

In [ ]:
df_h3 = df_h3_res

order = []
for col1 in MODEL_LIST: 
    for col2 in df_h3.columns:
        if col1 in col2:
            order.append(col2)
df_h3 = df_h3[order]

tab_df_h3 = make_latex_table(df_h3, decimals=4)
print(tab_df_h3)